In [1]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [2]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

### Goal: remove faulty documents
The goal is to remove faulty documents where mistakes were made when converting pdf into txt.

In [6]:
# load data
import ast
import pandas as pd

txtfiles = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")
# txtfiles['tokens'] = txtfiles['tokens'].apply(ast.literal_eval)
display(txtfiles.head())
df = txtfiles.copy()
df['clean_text'] = df['clean_tokens'].apply(lambda x: ' '.join(x))


In [7]:
display(df['clean_tokens_count'].describe())
class_describe = df.groupby('label')['clean_tokens_count'].describe()
display(class_describe)

count     33117.000000
mean       1392.480448
std        7326.313739
min           3.000000
25%         152.000000
50%         278.000000
75%         622.000000
max      143782.000000
Name: clean_tokens_count, dtype: float64

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
Actualiteit,996.0,360.197791,1833.846209,39.0,124.00,175.0,311.25,55400.0
Adviesaanvraag,2442.0,825.560197,1517.594501,33.0,287.00,486.0,818.00,39865.0
Agenda,3537.0,559.212327,2187.277067,17.0,180.00,337.0,515.00,72471.0
Amendement,1969.0,1274.853225,3332.397979,15.0,118.00,179.0,595.00,29968.0
Begroting,1967.0,7290.902898,26004.241417,24.0,125.00,203.0,1179.00,137501.0
Besluit,775.0,513.227097,889.155749,50.0,95.50,230.0,657.00,13651.0
Brief,1995.0,881.689724,954.436225,3.0,369.00,636.0,1134.00,21285.0
Factsheets,234.0,3347.893162,9863.108583,60.0,634.75,1527.0,2937.00,131816.0
Motie,8336.0,264.074976,584.938299,60.0,120.00,149.0,204.00,21285.0


##### Average tokens per page

In [11]:
df['tokens_page'] = df['clean_tokens_count']/df['num_pages']
display(df['tokens_page'].describe())
class_describe = df.groupby('label')['tokens_page'].describe()
display(class_describe)
display(df.head())

count    33114.000000
mean       146.037663
std         61.650676
min          3.000000
25%        104.750000
50%        135.000000
75%        180.000000
max       1744.500000
Name: tokens_page, dtype: float64

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
Actualiteit,996.0,130.756199,40.243522,39.000000,101.000000,127.000000,159.000000,307.000000
Adviesaanvraag,2442.0,184.328488,59.899469,33.000000,142.000000,181.666667,221.226966,794.000000
Agenda,3537.0,113.798911,33.555671,17.000000,96.000000,113.000000,130.500000,498.500000
Amendement,1968.0,145.302770,70.937087,7.500000,100.000000,129.000000,171.000000,758.000000
Begroting,1967.0,143.836542,60.184076,17.500000,103.000000,130.000000,172.200000,930.000000
Besluit,775.0,118.047598,43.077913,33.000000,83.525000,116.000000,147.000000,327.000000
Brief,1995.0,186.369043,42.818155,3.000000,156.333333,191.333333,217.200000,287.375000
Factsheets,234.0,229.006375,198.944347,38.000000,131.304688,193.312500,269.187500,1744.500000
Motie,8334.0,121.479216,38.506947,30.333333,94.000000,115.000000,140.000000,309.250000


,label,path,id,set,text,tokens,token_count,clean_tokens,clean_tokens_count,pdf_path,num_pages,clean_text,tokens_page
0,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,0,train,Gemeente Amsterdam\n% Gemeenteraad R\n% Gemeen...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",395,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",205,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,102.5
1,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,1,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",390,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",197,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,98.5
2,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,2,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",389,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",192,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,96.0
3,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,3,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",464,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",225,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,112.5
4,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,4,test,x Gemeente Amsterdam R\nGemeenteraad\n\n% Geme...,"[x, Gemeente, Amsterdam, R, Gemeenteraad, %, G...",261,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",134,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,134.0


#### Actualiteit

In [21]:
actualiteit = df.loc[df['label']=='Actualiteit']
threshold = actualiteit['tokens_page'].quantile(0.01)
lower_5_percentile_rows = actualiteit[actualiteit['tokens_page'] < threshold]
# print(len(lower_5_percentile_rows))
# display(lower_5_percentile_rows)
actualiteit['tokens_page'].describe()

for index, row in lower_5_percentile_rows.iterrows():
    print(row['clean_tokens_count'])
    print(row['text'])
    print('\n\n')

47
> Gemeente
Amsterdam

Commissieactualitert voor de Commissie KDD
van 23 Juni 2021
Portefeuille Diversiteit
Agendapunt Actualiteit
Onderwerp
Urgentie gewenst op aanwijzing 1 juli viering als nationale feestdag.
Aan de commissie
Kunst, Diversiteit en Democratisering
Reden van spoedeisendheid
Dit onderwerp moet in de commissie KDD van 23 juni besproken worden i.v.m. de naderende da-
tum van 1 juli. Als gewacht wordt tot na het reces, is die datum verstreken.
Het lid van de commissie
Nenita La Rose (PvdA)




46
> Gemeente

Amsterdam
Commissieactualiteit voor de commissie MOW
van 12 oktober 2022
Portefeuille Verkeer, Vervoer en Luchtkwaliteit
Agendapunt 2C
Onderwerp
Misbruik van gehandicaptenparkeerkaarten.
Zie: https:/fwww.at5.nlf/artikelen/217023/misbruik-gehandicaptenparkeerkaart-loopt-spuigaten-
vit-meerderheid-fraudeert
Aan de commissie
De fractie van het CDA heeft kennisgenomen van het bericht dat er mogelijk op grote schaal mis-
bruik wordt gemaakt van de gehandicaptenparkeerkaa